In [ ]:
!pip install transformers

     |████████████████████████████████| 2.2MB 7.7MB/s 
     |████████████████████████████████| 870kB 46.1MB/s 
     |████████████████████████████████| 3.3MB 50.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=403d363e9a6597d0f1ed98052dedfd47271e6735aba3fdc74a84271c15787da0
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
import torch
import torch.nn as nn 
from transformers import AutoTokenizer, BertTokenizer, EvalPrediction, AutoModel, BertPreTrainedModel, AutoConfig, BertModel

from transformers import BertForSequenceClassification
from transformers import AdamW
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForQuestionAnswering

from torch.utils.data import DataLoader

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pwd

/content


In [ ]:
circa_og = pd.read_csv('/content/gdrive/MyDrive/AMBER_TENG_NYU_DRIVE_TO_BROWN/NYU_SPRING2021/NLU/NLU_Project/circa-data.tsv', sep='\t',index_col='id')

In [ ]:
circa_og.head()

,context,question-X,canquestion-X,answer-Y,judgements,goldstandard1,goldstandard2
id,,,,,,,
0,Y has just travelled from a different city to ...,Are you employed?,I am employed .,I'm a veterinary technician.,Yes#Yes#Yes#Yes#Yes,Yes,Yes
1,X wants to know about Y's food preferences.,Are you a fan of Korean food?,I am a fan of Korean food .,I wouldn't say so,Probably no#No#No#No#Probably yes / sometimes yes,No,No
2,Y has just told X that he/she is thinking of b...,Are you bringing any pets into the flat?,I am bringing pets into the flat .,I do not own any pets,No#No#No#No#No,No,No
3,X wants to know what activities Y likes to do ...,Would you like to get some fresh air in your f...,I would like to get fresh air in my free time .,I am desperate to get out of the city.,"Yes#Yes, subject to some conditions#Probably y...",Yes,Yes
4,X and Y are childhood neighbours who unexpecte...,Is your family still living in the neighborhood?,My family is living in the neighborhood .,My parents are snowbirds now.,"No#In the middle, neither yes nor no#Probably ...","In the middle, neither yes nor no","In the middle, neither yes nor no"


In [ ]:
circa_og.shape

(34268, 7)

In [ ]:
circa_og.columns

Index(['context', 'question-X', 'canquestion-X', 'answer-Y', 'judgements',
       'goldstandard1', 'goldstandard2'],
      dtype='object')

In [ ]:
circa_og[circa_og['goldstandard1']=='I am not sure how X will interpret Y’s answer']

,context,question-X,canquestion-X,answer-Y,judgements,goldstandard1,goldstandard2
id,,,,,,,
878,Y has just travelled from a different city to ...,Do you stay up late?,I stay up late .,I usually stay up past 10pm.,Probably no#Probably yes / sometimes yes#I am ...,I am not sure how X will interpret Y’s answer,"In the middle, neither yes nor no"
908,X and Y are childhood neighbours who unexpecte...,Do you live close by?,I live close by .,My house is a few miles away.,I am not sure how X will interpret Y’s answer#...,I am not sure how X will interpret Y’s answer,"In the middle, neither yes nor no"
1422,Y has just told X that he/she is thinking of b...,Are you buying to rent?,I am buying to rent .,I'm just going to pay full price.,I am not sure how X will interpret Y’s answer#...,I am not sure how X will interpret Y’s answer,"In the middle, neither yes nor no"
1663,Y has just told X that he/she is thinking of b...,Are you buying to rent?,I am buying to rent .,I want to buy,No#Other#I am not sure how X will interpret Y’...,I am not sure how X will interpret Y’s answer,"In the middle, neither yes nor no"
1946,Y has just moved into a neighbourhood and meet...,Do you work near by?,I work near by .,I'll be at the store downtown.,"In the middle, neither yes nor no#In the middl...",I am not sure how X will interpret Y’s answer,"In the middle, neither yes nor no"
...,...,...,...,...,...,...,...
31174,Y has just told X that he/she is thinking of b...,"Do you make over $100,000 a year?","I make over $ 100,000 a year .",Please don't ask me that.,"In the middle, neither yes nor no#In the middl...",I am not sure how X will interpret Y’s answer,"In the middle, neither yes nor no"
32519,X wants to know what sorts of books Y likes to...,Do you like fantasy books as well?,I like fantasy books as well .,Fiction's better for me!,I am not sure how X will interpret Y’s answer#...,I am not sure how X will interpret Y’s answer,"In the middle, neither yes nor no"
33171,X and Y are childhood neighbours who unexpecte...,Is that really you?,That is I .,How rude!,I am not sure how X will interpret Y’s answer#...,I am not sure how X will interpret Y’s answer,"In the middle, neither yes nor no"


Strict-matched

In [ ]:
# circa_og = pd.read_csv('circa-data.tsv', sep='\t', index_col='id')
circa_s = circa_og.drop(circa_og.loc[circa_og['goldstandard1']=='Other'].index)
circa_s = circa_s.drop(circa_s.loc[circa_s['goldstandard1'].isnull()].index)
circa_s = circa_s.drop(circa_s.loc[circa_s['goldstandard1']=='I am not sure how X will interpret Y’s answer'].index)
strict = circa_s['goldstandard1']
YN_s = circa_s[['question-X','answer-Y']]

In [ ]:
YN_s

,question-X,answer-Y
id,,
0,Are you employed?,I'm a veterinary technician.
1,Are you a fan of Korean food?,I wouldn't say so
2,Are you bringing any pets into the flat?,I do not own any pets
3,Would you like to get some fresh air in your f...,I am desperate to get out of the city.
4,Is your family still living in the neighborhood?,My parents are snowbirds now.
...,...,...
34263,Do you like to drink?,I am in AA.
34264,Do you like pie?,My favorite pie is pecan.
34265,Want to go to a concert with me?,I'd rather do something else.


In [ ]:
strict.head()

id
0                                  Yes
1                                   No
2                                   No
3                                  Yes
4    In the middle, neither yes nor no
Name: goldstandard1, dtype: object

In [ ]:
circa_s.head()

,context,question-X,canquestion-X,answer-Y,judgements,goldstandard1,goldstandard2
id,,,,,,,
0,Y has just travelled from a different city to ...,Are you employed?,I am employed .,I'm a veterinary technician.,Yes#Yes#Yes#Yes#Yes,Yes,Yes
1,X wants to know about Y's food preferences.,Are you a fan of Korean food?,I am a fan of Korean food .,I wouldn't say so,Probably no#No#No#No#Probably yes / sometimes yes,No,No
2,Y has just told X that he/she is thinking of b...,Are you bringing any pets into the flat?,I am bringing pets into the flat .,I do not own any pets,No#No#No#No#No,No,No
3,X wants to know what activities Y likes to do ...,Would you like to get some fresh air in your f...,I would like to get fresh air in my free time .,I am desperate to get out of the city.,"Yes#Yes, subject to some conditions#Probably y...",Yes,Yes
4,X and Y are childhood neighbours who unexpecte...,Is your family still living in the neighborhood?,My family is living in the neighborhood .,My parents are snowbirds now.,"No#In the middle, neither yes nor no#Probably ...","In the middle, neither yes nor no","In the middle, neither yes nor no"


Relaxed-matched

In [ ]:
circa_r = circa_og.drop(circa_og.loc[circa_og['goldstandard2']=='Other'].index)
circa_r = circa_r.drop(circa_r.loc[circa_r['goldstandard2'].isnull()].index)
relaxed = circa_r['goldstandard2']
YN_r = circa_r[['question-X','answer-Y']]

Train, dev, test split (60/20/20)

In [ ]:
train_strict, val_strict, trainy_strict, valy_strict = train_test_split(YN_s, strict, test_size=.4)
test_strict, dev_strict, testy_strict, devy_strict = train_test_split(val_strict, valy_strict, test_size=.5)

In [ ]:
train_relaxed, val_relaxed, trainy_relaxed, valy_relaxed = train_test_split(YN_r, relaxed, test_size=.4)
test_relaxed, dev_relaxed, testy_relaxed, devy_relaxed = train_test_split(val_relaxed, valy_relaxed, test_size=.5)

In [ ]:
train_strict

,question-X,answer-Y
id,,
17607,Do you like where you're working?,Depends on the day of the week.
14582,Do you get along with everyone on this street?,I still haven't met everyone.
850,Do you know how expensive New York is?,The city is very expensive.
18356,Have you heard the latest music from Herb Alpert?,I have all his albums.
31874,Do you like oldies?,My car station is set to it.
...,...,...
10566,Do you fancy a drink after your long journey?,Fill up my glass.
28609,Have you traveled anywhere interesting?,I went to New Zealand a few years back.
32851,Have you read any books by Terry Pratchett?,I haven't heard of him


In [ ]:
trainy_strict.head()

id
17607      Yes, subject to some conditions
14582    In the middle, neither yes nor no
850                                    Yes
18356                                  Yes
31874                                  Yes
Name: goldstandard1, dtype: object

In [ ]:
test_strict.head()

,question-X,answer-Y
id,,
31382,Did you have a lot of meetings before?,We have a few every day -- it's hard to get re...
16509,Do you like bossanova?,What's that?
14751,Did you enjoy Taylor Swift's new album?,Is she still making music?
14406,Would you be open to joining some kind of club...,That sounds great.
33375,Have you ever taken a day off sick when you we...,My work ethic does not allow for this.


# Fine-Tuning QA using SQuAD 2.0

In [ ]:
! mkdir squad
! wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json -O squad/train-v2.0.json
! wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json -O squad/dev-v2.0.json

--2021-04-11 05:32:29--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘squad/train-v2.0.json’

squad/train-v2.0.js 100%[===================>]  40.17M   118MB/s    in 0.3s    

2021-04-11 05:32:30 (118 MB/s) - ‘squad/train-v2.0.json’ saved [42123633/42123633]

--2021-04-11 05:32:30--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘squad/dev-v2.0.json’

squ

In [ ]:
import json
from pathlib import Path

def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('squad/train-v2.0.json')
val_contexts, val_questions, val_answers = read_squad('squad/dev-v2.0.json')

In [ ]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [ ]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
        # if None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

In [ ]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [ ]:
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [ ]:

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()

model.eval()

KeyboardInterrupt: ignored

#Training on BERT

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-5)

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text_batch = 
encoding = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

SyntaxError: ignored

#Fine-Tuning BERT on Multi-NLI Dataset (Hugging Face)

In [ ]:
!pip install -U mxnet-cu101==1.7.0
!pip install d2l==0.16.2


     |███████████████████████████████▌| 834.1MB 1.6MB/s eta 0:00:08tcmalloc: large alloc 1147494400 bytes == 0x55c25a268000 @  0x7f3f3911c615 0x55c2209c806c 0x55c220aa7eba 0x55c2209cae8d 0x55c220abc99d 0x55c220a3efe9 0x55c220a39b0e 0x55c2209cc77a 0x55c220a3ee50 0x55c220a39b0e 0x55c2209cc77a 0x55c220a3b86a 0x55c220abd7c6 0x55c220a3aee2 0x55c220abd7c6 0x55c220a3aee2 0x55c220abd7c6 0x55c220a3aee2 0x55c220abd7c6 0x55c220a3aee2 0x55c2209cc69a 0x55c220a3ac9e 0x55c220a39e0d 0x55c2209cc77a 0x55c220a3aa45 0x55c2209cc69a 0x55c220a3aa45 0x55c220a39b0e 0x55c2209cc77a 0x55c220a3b86a 0x55c220a39b0e
     |████████████████████████████████| 846.0MB 20kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 81kB 5.0MB/s 


In [ ]:
import json
import multiprocessing
import os
from mxnet import gluon, np, npx
from mxnet.gluon import nn
from d2l import mxnet as d2l

npx.set_np()

In [ ]:
d2l.DATA_HUB['bert.base'] = (d2l.DATA_URL + 'bert.base.zip',
                             '7b3820b35da691042e5d34c0971ac3edbd80d3f4')
d2l.DATA_HUB['bert.small'] = (d2l.DATA_URL + 'bert.small.zip',
                              'a4e718a47137ccd1809c9107ab4f5edd317bae2c')

In [ ]:
def load_pretrained_model(pretrained_model, num_hiddens, ffn_num_hiddens,
                          num_heads, num_layers, dropout, max_len, devices):
    data_dir = d2l.download_extract(pretrained_model)
    # Define an empty vocabulary to load the predefined vocabulary
    vocab = d2l.Vocab()
    vocab.idx_to_token = json.load(open(os.path.join(data_dir, 'vocab.json')))
    vocab.token_to_idx = {
        token: idx for idx, token in enumerate(vocab.idx_to_token)}
    bert = d2l.BERTModel(len(vocab), num_hiddens, ffn_num_hiddens, num_heads,
                         num_layers, dropout, max_len)
    # Load pretrained BERT parameters
    bert.load_parameters(os.path.join(data_dir, 'pretrained.params'),
                         ctx=devices)
    return bert, vocab

In [ ]:
devices = d2l.try_all_gpus()
bert, vocab = load_pretrained_model('bert.small', num_hiddens=256,
                                    ffn_num_hiddens=512, num_heads=4,
                                    num_layers=2, dropout=0.1, max_len=512,
                                    devices=devices)

In [ ]:
class SNLIBERTDataset(gluon.data.Dataset):
    def __init__(self, dataset, max_len, vocab=None):
        all_premise_hypothesis_tokens = [[
            p_tokens, h_tokens] for p_tokens, h_tokens in zip(*[
                d2l.tokenize([s.lower() for s in sentences])
                for sentences in dataset[:2]])]

        self.labels = np.array(dataset[2])
        self.vocab = vocab
        self.max_len = max_len
        (self.all_token_ids, self.all_segments,
         self.valid_lens) = self._preprocess(all_premise_hypothesis_tokens)
        print('read ' + str(len(self.all_token_ids)) + ' examples')

    def _preprocess(self, all_premise_hypothesis_tokens):
        pool = multiprocessing.Pool(4)  # Use 4 worker processes
        out = pool.map(self._mp_worker, all_premise_hypothesis_tokens)
        all_token_ids = [token_ids for token_ids, segments, valid_len in out]
        all_segments = [segments for token_ids, segments, valid_len in out]
        valid_lens = [valid_len for token_ids, segments, valid_len in out]
        return (np.array(all_token_ids, dtype='int32'),
                np.array(all_segments, dtype='int32'), np.array(valid_lens))

    def _mp_worker(self, premise_hypothesis_tokens):
        p_tokens, h_tokens = premise_hypothesis_tokens
        self._truncate_pair_of_tokens(p_tokens, h_tokens)
        tokens, segments = d2l.get_tokens_and_segments(p_tokens, h_tokens)
        token_ids = self.vocab[tokens] + [self.vocab['<pad>']] \
                             * (self.max_len - len(tokens))
        segments = segments + [0] * (self.max_len - len(segments))
        valid_len = len(tokens)
        return token_ids, segments, valid_len

    def _truncate_pair_of_tokens(self, p_tokens, h_tokens):
        # Reserve slots for '<CLS>', '<SEP>', and '<SEP>' tokens for the BERT
        # input
        while len(p_tokens) + len(h_tokens) > self.max_len - 3:
            if len(p_tokens) > len(h_tokens):
                p_tokens.pop()
            else:
                h_tokens.pop()

    def __getitem__(self, idx):
        return (self.all_token_ids[idx], self.all_segments[idx],
                self.valid_lens[idx]), self.labels[idx]

    def __len__(self):
        return len(self.all_token_ids)

In [ ]:
# Reduce `batch_size` if there is an out of memory error. In the original BERT
# model, `max_len` = 512
batch_size, max_len, num_workers = 512, 128, d2l.get_dataloader_workers()
data_dir = d2l.download_extract('SNLI')
train_set = SNLIBERTDataset(d2l.read_snli(data_dir, True), max_len, vocab)
test_set = SNLIBERTDataset(d2l.read_snli(data_dir, False), max_len, vocab)
train_iter = gluon.data.DataLoader(train_set, batch_size, shuffle=True,
                                   num_workers=num_workers)
test_iter = gluon.data.DataLoader(test_set, batch_size,
                                  num_workers=num_workers)

In [ ]:
class BERTClassifier(nn.Block):
    def __init__(self, bert):
        super(BERTClassifier, self).__init__()
        self.encoder = bert.encoder
        self.hidden = bert.hidden
        self.output = nn.Dense(3)

    def forward(self, inputs):
        tokens_X, segments_X, valid_lens_x = inputs
        encoded_X = self.encoder(tokens_X, segments_X, valid_lens_x)
        return self.output(self.hidden(encoded_X[:, 0, :]))